In [30]:
from pyleri import Grammar, Keyword, Sequence, Choice, Token

In [48]:
class MyGrammar(Grammar):
    """
    base noun, number (pl, sg), conjugate (nom, gen, dat, acc, ins, loc, voc), gender (m, f, n)
    base adjective, number (pl, sg), conjugate (nom, gen, dat, acc, ins, loc, voc), gender (m, f, n)
    base verb, number (pl, sg), conjugate (nom, gen, dat, acc, ins, loc, voc), gender (m, f, n), person (1,2,3), time (past, pres, futu), mode (ind, pre, imp)
    """
    ZAMEK_SG_NOM = Keyword('zamek')
    ZAMEK_SG_ACC = Keyword('zamek')
    ZAMEK_PL_ACC = Keyword('zamki')
    ALA_SG_ACC = Keyword('Alę')
    MIEC_PRES_3SG = Keyword('ma')
    object_sg_nom = Choice(ZAMEK_SG_NOM)
    object_sg_acc = Choice(ZAMEK_SG_ACC, ALA_SG_ACC)
    object_pl_acc = Choice(ZAMEK_PL_ACC)
    subject_sg = Choice(object_sg_nom)
    object_acc = Choice(object_sg_acc, object_pl_acc)
    verb_sg_acc = Choice(MIEC_PRES_3SG)
    START = Sequence(subject_sg, verb_sg_acc, object_acc)

# Compile your grammar by creating an instance of the Grammar Class.
my_grammar = MyGrammar()

print(my_grammar.parse('zamek ma zamek').as_str()) # => error at position 0, expecting: hi

parsed successfully


In [62]:
import tkinter as tk
from tkinter import font

# INTERESTING return type("Result", (), {"pos": None, "expecting": {"I", "like", "apples", "and", "bananas"}})()


class GrammarEditor:
    def __init__(self, root):
        self.root = root
        self.root.title("Grammar Learning Editor")
        self.text = tk.Text(root, wrap="word", undo=True)
        self.text.pack(fill="both", expand=True)
        self.text.focus_set()

        self.text.bind("<KeyRelease>", self.on_text_change)
        self.text.bind("<Tab>", self.on_tab)
        self.root.bind_all("<Up>", self.on_up, add="+")
        self.root.bind_all("<Down>", self.on_down, add="+")
        self.text.tag_configure("error", underline=True, foreground="red")

        # Popup for suggestions
        self.popup = tk.Listbox(root, height=5)
        self.popup.bind("<Double-Button-1>", self.apply_suggestion)
        self.popup.bind("<Return>", self.apply_suggestion)
        self.popup_is_visible = False
        
        self.selected: int = 0
        self.suggestions = []
        self.error_pos = None

    def on_text_change(self, event=None):
        text = self.text.get("1.0", "end-1c")
        result = my_grammar.parse(text)

        # Clear error tag
        self.text.tag_remove("error", "1.0", "end")

        # Hide popup if not needed
        if len(result.expecting) == 0:
            self.hide_popup()
            return
        
        self.error_pos = result.pos
        self.suggestions = [str(elem) for elem in result.expecting]

        # Get text index for red underline
        index = self.text.index(f"1.0+{result.pos}c")
        next_space = self.text.search(r"\s", index, regexp=True)
        if not next_space:
            next_space = "end"

        self.text.tag_add("error", index, next_space)
        self.show_popup(index, self.suggestions)

    def show_popup(self, index, suggestions):
        bbox = self.text.bbox(index)
        if not bbox:
            self.hide_popup()
            return

        x, y, width, height = bbox
        self.popup.place(x=x, y=y + height)
        self.popup.delete(0, tk.END)
        for s in suggestions:
            self.popup.insert(tk.END, s)
        self.popup_is_visible = True
        self.popup.selection_clear(0, tk.END)
        self.popup.selection_set(self.selected)

    def hide_popup(self):
        self.popup.place_forget()
        self.popup_is_visible = False

    def on_tab(self, event):
        if self.popup_is_visible and self.suggestions:
            suggestion = self.suggestions[self.popup.curselection()[0]]
            self.insert_suggestion(suggestion)
            return "break"  # Prevent default tab
        return None
    
    def on_up(self, event):
        """Navigate popup list upward."""
        if not self.popup_is_visible:
            return None
        cur = self.popup.curselection()
        if not cur:
            new = 0
        else:
            new = (cur[0] - 1) % self.popup.size()
        self.selected = new
        self.popup.selection_clear(0, tk.END)
        self.popup.selection_set(new)
        self.popup.activate(new)
        return "break"

    def on_down(self, event):
        """Navigate popup list downward."""
        if not self.popup_is_visible:
            return None
        cur = self.popup.curselection()
        if not cur:
            new = 0
        else:
            new = (cur[0] + 1) % self.popup.size()
        self.selected = new
        self.popup.selection_clear(0, tk.END)
        self.popup.selection_set(new)
        self.popup.activate(new)
        return "break"

    def apply_suggestion(self, event):
        if self.popup_is_visible:
            suggestion = self.suggestions[self.popup.curselection()[0]]
            self.insert_suggestion(suggestion)

    def insert_suggestion(self, suggestion):
        if self.error_pos is not None:
            start = self.text.index(f"1.0+{self.error_pos}c")
            next_space = self.text.search(r"\s", start, regexp=True)
            if not next_space:
                next_space = "end"
            self.text.delete(start, next_space)
            self.text.insert(start, suggestion + " ")
            self.hide_popup()
            self.on_text_change()


if __name__ == "__main__":
    root = tk.Tk()
    app = GrammarEditor(root)
    root.mainloop()
